In [2]:
import pandas as pd

pd.set_option('display.max_rows', None)        # Show all rows
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.width', None)           # Disable line wrapping
pd.set_option('display.max_colwidth', None)    # Show full column content (especially for strings)

def load_and_sort_csv(path):
    df = pd.read_csv(path, parse_dates=['Date'], index_col='Date')  # parse 'date' as datetime
    df = df.sort_index(ascending=True)  # sort by date
    # df.index = pd.to_datetime(df.index)
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

In [3]:
cbbi = load_and_sort_csv('../data/processed/Bitcoin Bull Run Index (CBBI).csv')
# add postfix _cbbi before the column names
cbbi.columns = [f"{col}_cbbi" for col in cbbi.columns]
# cscsi20 = load_and_sort_csv('../data/processed/cscsi20_CompassSESAMmCryptoSentimentIndex.csv')
dataset = load_and_sort_csv('../data/processed/dataset.csv')
fear_greed = load_and_sort_csv('../data/processed/fear_greed_index.csv')
sentiment = load_and_sort_csv('../data/processed/sentiment_grouped.csv')

# Check for missing dates in each dataframe
print("Date ranges and missing dates:")
for name, df in [('CBBI', cbbi), ('Dataset', dataset), 
                ('Fear & Greed', fear_greed), ('Sentiment', sentiment)]:
    date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    missing_dates = date_range.difference(df.index)
    print(f"\n{name}:")
    print(f"Date range: {df.index.min()} to {df.index.max()}")
    # print(f"Missing dates: {len(missing_dates)}")
    # if len(missing_dates) > 0:
    #     print(f"First few missing dates: {missing_dates[:5]}")

Date ranges and missing dates:

CBBI:
Date range: 2011-06-27 00:00:00 to 2025-04-21 00:00:00

Dataset:
Date range: 2016-12-06 00:00:00 to 2025-05-11 00:00:00

Fear & Greed:
Date range: 2011-01-03 00:00:00 to 2025-05-06 00:00:00

Sentiment:
Date range: 2016-11-01 00:00:00 to 2025-01-13 00:00:00


In [4]:
# Find mutual date range across all dataframes
start = max(df.index.min() for df in [cbbi, dataset, fear_greed, sentiment])
end = min(df.index.max() for df in [cbbi, dataset, fear_greed, sentiment])
calendar = pd.date_range(start=start, end=end, freq='D')

# Reindex each to the predefined calendar
cbbi        = cbbi.reindex(calendar)
dataset     = dataset.reindex(calendar)
fear_greed  = fear_greed.reindex(calendar)
sentiment   = sentiment.reindex(calendar)

# Forward fill missing values
for df in [cbbi, dataset, fear_greed, sentiment]:
    df.ffill(inplace=True)

# Inner join all dataframes
df = pd.concat([cbbi, dataset, fear_greed, sentiment],
               axis=1,
               join='inner')

# Move BTC column to end
btc_col = 'btc_price' if 'btc_price' in df.columns else 'btc_close'
df = pd.concat([df.drop(columns=[btc_col]), df[btc_col]], axis=1)

# Export with 'Date' as a column
df_reset = df.reset_index().rename(columns={'index': 'Date'})
df_reset.to_csv('../data/final/final_dataset.csv', index=False)
print("Result shape:", df.shape)

Result shape: (2961, 62)


In [5]:
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
missing_dates = date_range.difference(df.index)
print(f"\n{name}:")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Missing dates: {len(missing_dates)}")
if len(missing_dates) > 0:
    print(f"First few missing dates: {missing_dates[:5]}")


Sentiment:
Date range: 2016-12-06 00:00:00 to 2025-01-13 00:00:00
Missing dates: 0
